In [3]:
# Líneas de código para subir los archivos a la carpeta Colab.
from google.colab import files
uploades = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [4]:
# Carga de los paquetes necesarios.
# Paquete para cargar data frames.
import pandas as pd
# Paquetes para generar el SVM bagging
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
import statistics
from statistics import mean

In [5]:
# Carga de los archivos csv.
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

In [6]:
# Vemos el formato en el que se han importado los datos. Observamos que se ha introducido una columna adicional que informa del número de la entrada en las filas.
print(test.head())

# Para eliminarla, utilizamos la función drop.
test = test.drop("Unnamed: 0", axis = 1)
train = train.drop("Unnamed: 0", axis = 1)

                             Unnamed: 0  ENSG00000002726.21  \
0  0c955374-5e40-4d22-b9a5-1b7d3dcb444f            0.000714   
1  196e50f4-d4f1-4070-a515-11c1876de6a8            0.174161   
2  1d47d8d2-d45c-4376-a434-c1d18a357e8b            0.114204   
3  266ec37d-2981-4e70-88b5-2c864fb1fca4            0.001428   
4  2790ee15-f7ac-41c5-b74c-ef436638d94d            0.077088   

   ENSG00000007312.13  ENSG00000008311.15  ENSG00000008394.13  \
0            0.001009            0.005229            0.002362   
1            0.021733            0.135730            0.125796   
2            0.033435            0.148366            0.070407   
3            0.000662            0.006318            0.005484   
4            0.023626            0.095425            0.115250   

   ENSG00000010278.15  ENSG00000010610.10  ENSG00000011600.12  \
0            0.014665            0.054327            0.039849   
1            0.013491            0.108640            0.061171   
2            0.013271            0.

In [7]:
# Reevaluamos.
print(test.head())

   ENSG00000002726.21  ENSG00000007312.13  ENSG00000008311.15  \
0            0.000714            0.001009            0.005229   
1            0.174161            0.021733            0.135730   
2            0.114204            0.033435            0.148366   
3            0.001428            0.000662            0.006318   
4            0.077088            0.023626            0.095425   

   ENSG00000008394.13  ENSG00000010278.15  ENSG00000010610.10  \
0            0.002362            0.014665            0.054327   
1            0.125796            0.013491            0.108640   
2            0.070407            0.013271            0.106141   
3            0.005484            0.010261            0.069148   
4            0.115250            0.009894            0.160907   

   ENSG00000011600.12  ENSG00000012124.17  ENSG00000014257.16  \
0            0.039849            0.003668            0.069712   
1            0.061171            0.059131            0.521521   
2            0.045394  

In [8]:
# Vamos a separar los datos en las clases y en sus correspondendientes datos. Trasformamos los datos de train y test en matrices usando la función .values.
test_x = test.values
train_y = train['clase']
train_x = train.drop('clase', axis = 1).values

In [9]:
# Ahora vamos a generar el bagging classifier de SVM. Como hemos visto que el que
# mejor funcionaba era el SVM Linear, vamos a usar este kernel. Primero establecemos el método que se va a usar como estimador.
np.random.seed(21081999)
svm = LinearSVC(C=1.0)
# Generamos el modelo de Bagging con el SVM. Le decimos que haga 20 estimaciones.
model = BaggingClassifier(estimator = svm, n_estimators= 50)
# Generamos la repeated k-fold cross-validation como hacíamos en R. Usamos los mismos parámetros.
rkf = RepeatedKFold(n_splits = 10, n_repeats= 10, random_state=1)
# Extraemos las métricas del modelo para cada valor del repeated-CV.
scores = cross_val_score(model, train_x, train_y, scoring="accuracy", cv = rkf, n_jobs=-1)
print(mean(scores))

0.9965804132973944


In [10]:
# Ajustamos el modelo.
model.fit(train_x, train_y)
# Utilizamos el ajuste del modelo para predecir.
y_pred = model.predict(test_x)
# Exportamos las clases predichas para terminar el análisis en R.
df_ypred = pd.DataFrame(y_pred)
df_ypred.to_csv('ypred.csv')